In [5]:
print("ok")

ok


In [6]:
%pwd

'd:\\work\\Githubdeskwork\\Medical-Chatbot\\research'

In [7]:
import os
os.chdir('../')

In [8]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter

In [9]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents


In [10]:
extracted_data = load_pdf_file(data="Data/")

In [11]:
extracted_data

[Document(metadata={'producer': 'Atop CHM to PDF Converter', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'moddate': '2014-04-21T07:53:19+10:00', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'source': 'Data\\Merck Sharp & Dohme Corp. - The Merck Manual of Diagnosis & Therapy, 19th Edition (2012) - libgen.li.pdf', 'total_pages': 4114, 'page': 0, 'page_label': 'i'}, page_content=''),
 Document(metadata={'producer': 'Atop CHM to PDF Converter', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'moddate': '2014-04-21T07:53:19+10:00', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'source': 'Data\\Merck Sharp & Dohme Corp. - The Merck Manual of Diagnosis & Therapy, 19th Edition (2012) - libgen.li.pdf', 'total_pages': 4114, 'page': 1, 'page_label': 'ii'}, page_content=''),
 Document(metadata={'producer': 'Atop CHM to PDF Converter', 'creator': 'Atop CHM to PDF Converter', 'creat

In [12]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [13]:
text_chunks = text_split(extracted_data)
print("Length of text_chunks: ", len(text_chunks))

Length of text_chunks:  30465


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [12]:
embeddings=download_hugging_face_embeddings()

C:\Users\acer\AppData\Local\Temp\ipykernel_13904\697957399.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\acer\.conda\envs\mbt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
query_result=embeddings.embed_query("I am a good man")
print("Query Result: ", query_result)
# print("Length of Query Result: ", len(query_result))

Query Result:  [-0.05281347408890724, -0.001374902087263763, 0.017714640125632286, -0.041204363107681274, -0.07703106105327606, -0.012953074648976326, 0.08606577664613724, -0.035605158656835556, -0.007505625952035189, 0.011056408286094666, -0.00831371545791626, -0.03916935622692108, -0.010008127428591251, -0.013129576109349728, 0.05076593533158302, -0.010774414986371994, -0.012444491498172283, -0.008539211936295033, 0.026145555078983307, -0.006662552244961262, -0.0873192623257637, 0.05264405533671379, 0.013883042149245739, -0.05309411138296127, -0.10256103426218033, -0.002758856862783432, -0.012859216891229153, 0.01733781024813652, 0.030651090666651726, 0.012479221448302269, -0.05459626391530037, 0.02546037547290325, 0.06263039261102676, 0.005438545253127813, -0.029814807698130608, -0.024091437458992004, -0.036041129380464554, -0.060643650591373444, 0.04598749056458473, 0.05024239420890808, 0.010332739911973476, -0.03231607377529144, 0.013480001129209995, -0.008511165156960487, 0.00642

In [25]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
print("PINECONE_API_KEY: ", PINECONE_API_KEY)

PINECONE_API_KEY:  pcsk_TfmYV_7TQz5LhQ7oYx33ZSpAjYY1Kgnzvnaz9bSZFiJqK9eYGva1wuiFC8xRBEMa9hYNm


In [26]:
import os
from pinecone import Pinecone, ServerlessSpec

# Retrieve the API key from your environment or set it directly
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

# Create an instance of the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medbot"

# Optionally, check if the index already exists before creating it
existing_indexes = [index.name for index in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


In [27]:
import os
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY

In [29]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
    batch_size=50
)


In [30]:
docsearch

In [13]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore

index_name = "medbot"

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [14]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})


In [32]:
retrieved_docs = retriever.invoke("What is acne?")

In [33]:
retrieved_docs

[Document(id='5617d694-03ce-4865-b886-f147020f4084', metadata={'creationdate': '2012-06-15T05:44:40+00:00', 'creator': 'Atop CHM to PDF Converter', 'moddate': '2014-04-21T07:53:19+10:00', 'page': 769.0, 'page_label': '760', 'producer': 'Atop CHM to PDF Converter', 'source': 'Data\\Merck Sharp & Dohme Corp. - The Merck Manual of Diagnosis & Therapy, 19th Edition (2012) - libgen.li.pdf', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'total_pages': 4114.0}, page_content='The Merck Manual of Diagnosis & Therapy, 19th Edition Chapter 73. Acne & Related Disorders\n760'),
 Document(id='ce67ffa7-04cb-4b4f-a4ef-5f14dceb7f1f', metadata={'creationdate': '2012-06-15T05:44:40+00:00', 'creator': 'Atop CHM to PDF Converter', 'moddate': '2014-04-21T07:53:19+10:00', 'page': 769.0, 'page_label': '760', 'producer': 'Atop CHM to PDF Converter', 'source': 'Data\\Merck Sharp & Dohme Corp. - The Merck Manual of Diagnosis & Therapy, 19th Edition (2012) - libgen.li.pdf', 'title': 'The Merck

In [ ]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4,max_tokens=500)

In [2]:
import os

gemini_api_key = os.environ['GEMINI_API_KEY']

In [3]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", api_key=gemini_api_key)

In [19]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.4, max_output_tokens=500,api_key=gemini_api_key)

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
  ]
)

In [21]:
question_answering_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [22]:
response = rag_chain.invoke({"input":"What is acne?"})
print(response["answer"])

Acne vulgaris is the formation of comedones, papules, pustules, nodules, and/or cysts. It results from obstruction and inflammation of pilosebaceous units. It most often affects adolescents.


In [18]:
import google.generativeai as genai
import os

# Make sure you have your API key configured
genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))

for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
models/gemma-3-27b-i

In [23]:
response = rag_chain.invoke({"input":"What is stats?"})
print(response["answer"])

I am sorry, but the provided text does not contain information about the field of statistics. The text discusses IQ levels, leukocyte esterase tests, and statin use, but it does not define or explain "stats."


In [24]:
response = rag_chain.invoke({"input":"Acromegaly"})
print(response["answer"])

Acromegaly is a syndrome caused by excessive growth hormone secretion, typically due to a pituitary adenoma. Diagnosis involves clinical evaluation, skull and hand x-rays, and measurement of growth hormone levels that are not suppressed by glucose administration. Treatment focuses on removing or destroying the adenoma.
